In [8]:
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import sessionmaker

import sys
sys.path.append('..')

from tracks_interactions.db.db_model import CellDB, TrackDB,Base
from tracks_interactions.db.db_functions import get_descendants

In [9]:
# define pathways

org_db_path = r'D:\kasia\tracking\E6_exp\double_segmentation_ultrack\Exp6_gardener.db'

new_db_path = r'../tests/fixtures/db_2tables_test.db'

In [10]:
engine = create_engine(f'sqlite:///{org_db_path}')
session = sessionmaker(bind=engine)()

new_engine = create_engine(f'sqlite:///{new_db_path}')
Base.metadata.create_all(new_engine)
new_session = sessionmaker(bind=new_engine)()

In [11]:
# get all the descendants of the selected roots
# don't ask for not roots as their parameters will be incorrect in the test database
selected_roots = [37401,20422]
selected_track_id_list = []

for root in selected_roots:
    selected_track_id_list.extend([x.track_id for x in get_descendants(session,root)])

selected_track_id_list

[37401, 37402, 37403, 20422, 20423, 20424, 20425, 20426, 20427, 20428]

In [12]:
# copy the selected tracks and cells to the new database

for selected_track_id in selected_track_id_list:
    
    cells = session.query(CellDB).filter(CellDB.track_id == selected_track_id).all()
    tracks = session.query(TrackDB).filter(TrackDB.track_id == selected_track_id).all()

    for cell in cells:
        
        # Create a new instance of CellDB
        new_cell = CellDB()
        
        # Deep copy
        for key, value in inspect(cell).attrs.items():
            setattr(new_cell, key, value.value)

        new_session.add(new_cell)

    for track in tracks:

        # Create a new instance of TrackDB
        new_track = TrackDB()

        # Deep copy
        for key, value in inspect(track).attrs.items():
            setattr(new_track, key, value.value)
        
        new_session.add(new_track)

    new_session.commit()

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: cells.track_id, cells.t
[SQL: INSERT INTO cells (track_id, t, id, "row", col, bbox_0, bbox_1, bbox_2, bbox_3, mask, signals, tags) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: ((37401, 21, 22006648, 5160, 5353, 5146, 5343, 5174, 5363, <memory at 0x000002857C40D3C0>, '{"area": 437, "ch0_nuc": 127.89931350114416, "ch0_cyto": 123.99120879120879, "ch1_nuc": 1602.4324942791761, "ch1_cyto": 174.16483516483515}', '{}'), (37401, 22, 23006707, 5157, 5353, 5144, 5343, 5172, 5362, <memory at 0x000002857C40FC40>, '{"area": 408, "ch0_nuc": 128.52941176470588, "ch0_cyto": 124.37914691943128, "ch1_nuc": 1741.7671568627452, "ch1_cyto": 182.17298578199052}', '{}'), (37401, 23, 24006875, 5158, 5352, 5144, 5343, 5173, 5363, <memory at 0x000002857C40FF40>, '{"area": 433, "ch0_nuc": 128.30715935334874, "ch0_cyto": 124.59009009009009, "ch1_nuc": 1688.715935334873, "ch1_cyto": 178.5045045045045}', '{}'), (37401, 24, 25007036, 515

In [7]:
# clean up
session.close()
new_session.close()

In [14]:
# get all attributes of the cell
cell.__dict__.keys()
# get all not hidden attributes of the cell 


dict_keys(['_sa_instance_state', 'col', 't', 'id', 'bbox_1', 'bbox_3', 'signals', 'track_id', 'row', 'bbox_0', 'bbox_2', 'mask', 'tags'])

In [18]:
[x[0] for x in inspect(cell).attrs.items() if not x[0] in ['id']]

['track_id',
 't',
 'row',
 'col',
 'bbox_0',
 'bbox_1',
 'bbox_2',
 'bbox_3',
 'mask',
 'signals',
 'tags']